In [38]:
import os
import numpy as np
import networkx as nx
import csv
import time
import pandas as pd
import copy

## 課題２.  Heap搭載Dijkstraを作って，heap非搭載と速度比較

---

### まずheapクラスを，ノード番号も持てるように改良

In [26]:
class heap:
    def __init__(self, node_list):# ここでの node_list は（コスト，ノード番号）の２要素タプルを想定．
        self.h_list = []
        length = len(node_list)
        for i in range(length):
            self.heappush(node_list[i])
            # print(self.h_list)
    
    
    def swap(self, j, k):
        self.h_list[j], self.h_list[k] = self.h_list[k], self.h_list[j]
    
    
    def heappush(self, item):
        self.h_list.append(item)
        length = len(self.h_list)
        son = length - 1
        while True:
            par = (son - 1)//2
            if par < 0:
                break
            elif self.h_list[son][0] < self.h_list[par][0]:# コストのみ見て並べ替え
                self.swap(son, par)
                son = par
            else:
                break

    
    def heappop(self):
        tail = self.h_list.pop(-1)
        if self.h_list == []:
            return tail
        
        # 最後尾要素を先頭へ
        pop_item       = self.h_list[0]
        self.h_list[0] = tail
        
        length = len(self.h_list)
        
        # 上からヒープ作り直し
        par = 0
        while True:
            son = par*2 + 1
            if son >= length:
                break
            elif son+1 == length:
                if self.h_list[son][0] < self.h_list[par][0]:
                    self.swap(son, par)
                    par = son
                else:
                    break
            else:
                if self.h_list[son][0] < self.h_list[son+1][0]:
                    if self.h_list[son][0] < self.h_list[par][0]:
                        self.swap(son, par)
                        par = son
                    else:
                        break
                else:
                    if self.h_list[son+1][0] < self.h_list[par][0]:
                        self.swap(son+1, par)
                        par = son + 1
                    else:
                        break
            
        return pop_item

In [27]:
a = [(1, 1), (9, 9), (5, 5), (2, 2), (8, 8), (10, 10), (3, 3), (4, 4), (7, 7), (6, 6)]

In [28]:
h = heap(a)

In [29]:
h.heappush((5, 5))

In [30]:
h.h_list

[(1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (10, 10),
 (5, 5),
 (9, 9),
 (7, 7),
 (8, 8),
 (6, 6)]

In [31]:
while h.h_list:
    print(h.heappop())
    print(h.h_list)

(1, 1)
[(2, 2), (4, 4), (3, 3), (6, 6), (5, 5), (10, 10), (5, 5), (9, 9), (7, 7), (8, 8)]
(2, 2)
[(3, 3), (4, 4), (5, 5), (6, 6), (5, 5), (10, 10), (8, 8), (9, 9), (7, 7)]
(3, 3)
[(4, 4), (5, 5), (5, 5), (6, 6), (7, 7), (10, 10), (8, 8), (9, 9)]
(4, 4)
[(5, 5), (5, 5), (8, 8), (6, 6), (7, 7), (10, 10), (9, 9)]
(5, 5)
[(5, 5), (6, 6), (8, 8), (9, 9), (7, 7), (10, 10)]
(5, 5)
[(6, 6), (7, 7), (8, 8), (9, 9), (10, 10)]
(6, 6)
[(7, 7), (9, 9), (8, 8), (10, 10)]
(7, 7)
[(8, 8), (9, 9), (10, 10)]
(8, 8)
[(9, 9), (10, 10)]
(9, 9)
[(10, 10)]
(10, 10)
[]


### Dijkstra with heap

In [33]:
# この書き方は flag という変数を使って，ノードを（未探索，探索中，探索済）という３状態に区別している．
# 元の書き方は（未確定，確定済）の２状態だったが，３状態にすればヒープは探索中のみを対象にすればよく，サイズが小さくて済む．

def dijkstra(N, L, origin):
    INF = 10 ** 9
    
    num_node = len(N)
    num_link = len(L)
    
    spl  = np.ones(shape=num_node)*INF#shortest path length
    flag = np.zeros(shape=num_node)
    
    hq = heap([(0, origin)]) # (distance, node)
    spl[N[origin]['index']]  = 0
    
    while hq.h_list:
        from_node = hq.heappop()[1] # 確定させるノードを pop
            
        if flag[N[from_node]['index']] == 2:
            continue #探索済ならスキップ
        else:
            for l in N[from_node]['outlink_list']:
                to_node = L[l]['to_node']
                tmp_cost = spl[N[from_node]['index']] + L[l]['ff_time']
            
                if flag[N[to_node]['index']] == 2:
                    continue #探索済ならスキップ
                else:
                    if tmp_cost < spl[N[to_node]['index']]:
                        spl[N[to_node]['index']]  = tmp_cost
                        flag[N[to_node]['index']] = 1 #探索中に変更
                        hq.heappush((tmp_cost, to_node)) #ヒープに追加
                
            flag[N[from_node]['index']] = 2 #探索済に変更 
        
    return spl

### without heap

In [34]:
# 元の書き方

def dijkstra_old(N, L, Origin):
    origin = Origin - 1
    
    # 無限大の代わり
    inf = 10 ** 9
    
    nodeCosts = {}

    # non-determinedなノードのリスト
    nodesNondet = {}

    # ノードコストの初期化
    for i in range(len(N)):
        node=N[i+1]['index']  
        nodeCosts[node]=inf
        nodesNondet[node]=True
 

    # 起点ノードのノードコストを決める
    nodeCosts[origin] = 0

    while True:
        # non-determinedのノードの中から最小のノードコストを持つものを探す
        minCost = inf
        minCostNode = ''
        for node in nodesNondet.keys():
            if minCost > nodeCosts[node]:
                minCost = nodeCosts[node]
                minCostNode = node
        
        # 涌井が追加，もうたどり着けるノードがなければbreak
        if minCostNode == '':
            break
  
        # 最小ノードコストを持つノードをdeterminedにする
        # print ('最小ノードコストのノード:',minCostNode,'そのコスト:',minCost)
        del nodesNondet[minCostNode]

        # non-determinedのノードがなければおしまい
        if len(nodesNondet) == 0:
            break
  
        # 最小ノードコストのノードの隣接ノードのノードコストを更新する
        for nextLink in N[minCostNode+1]['outlink_list']:
            newCost = minCost + L[nextLink]['ff_time']
            nextNode=L[nextLink]['to_node'] - 1
            if newCost < nodeCosts[nextNode]:
                nodeCosts[nextNode] = newCost

    # print ("---------------")
    # print ("計算終了")
    return nodeCosts

In [11]:
N = {} #ノードの情報
L = {} #リンクの情報

# f = open("../GitHub/data_set/tntp_box/ChicagoSketch_net.tntp", "r")
f = open("../GitHub/data_set/tntp_box/Sydney_net.tntp", "r")

flag = False
i = 0
for line in f:# ここでファイルを一行ずつ読んでいる．一気に読まないので使用メモリが少ない（多分）
    if flag == True:
        link_parameter = line.split()# 一つの文字列を，スペースとか改行とかタブとかで分割する
    
        from_node = int(link_parameter[0])
        to_node = int(link_parameter[1])
        ff_time = float(link_parameter[4])
        
        L[i+1] = {}
        L[i+1]["index"]     = i
        L[i+1]["from_node"] = from_node
        L[i+1]["to_node"]   = to_node
        L[i+1]["ff_time"]   = ff_time
        
        if N.get(from_node):# Nの中に from_node があるなら
            N[from_node]['outlink_list'] = N[from_node]['outlink_list'] + [i+1]
        else:
            N[from_node] = {}
            N[from_node]["index"] = from_node-1
            N[from_node]['outlink_list'] = [i+1]
            N[from_node]['inlink_list']  = []
            
        if N.get(to_node):# Nの中に to_node があるなら
            N[to_node]['inlink_list'] = N[to_node]['inlink_list'] + [i+1]
        else:
            N[to_node] = {}
            N[to_node]["index"] = to_node-1
            N[to_node]['outlink_list'] = []
            N[to_node]['inlink_list']  = [i+1]
            
        i = i + 1
    
    
    if line[0] == "~":
        flag = True
    
    
f.close()
len(N)

33113

In [25]:
%%time
dijkstra(N, L, 1)

Wall time: 330 ms


array([0.000e+00, 1.000e+09, 3.600e+00, ..., 4.187e+01, 6.110e+00,
       1.126e+01])

In [16]:
%%time
dijkstra_old(N, L, 1)

Wall time: 39.9 s


{0: 0,
 1: 1000000000,
 2: 3.5999999999999996,
 3: 4.14,
 4: 4.47,
 5: 3.04,
 6: 2.63,
 7: 3.0500000000000003,
 8: 2.88,
 9: 3.32,
 10: 3.5999999999999996,
 11: 3.8499999999999996,
 12: 3.4499999999999997,
 13: 2.4299999999999997,
 14: 2.76,
 15: 2.9499999999999997,
 16: 3.46,
 17: 3.7999999999999994,
 18: 3.6399999999999997,
 19: 3.6499999999999995,
 20: 3.33,
 21: 2.46,
 22: 2.6799999999999997,
 23: 2.99,
 24: 3.27,
 25: 2.98,
 26: 3.26,
 27: 3.3899999999999997,
 28: 3.21,
 29: 3.23,
 30: 3.34,
 31: 3.3,
 32: 3.4299999999999997,
 33: 3.3899999999999997,
 34: 3.55,
 35: 3.4899999999999998,
 36: 3.6699999999999995,
 37: 5.519999999999999,
 38: 6.099999999999998,
 39: 3.52,
 40: 3.61,
 41: 3.8099999999999996,
 42: 3.78,
 43: 3.78,
 44: 3.769999999999999,
 45: 3.9799999999999995,
 46: 3.6799999999999997,
 47: 4.01,
 48: 4.1499999999999995,
 49: 3.94,
 50: 4.04,
 51: 4.23,
 52: 4.37,
 53: 3.98,
 54: 4.45,
 55: 4.32,
 56: 4.209999999999999,
 57: 4.01,
 58: 4.1899999999999995,
 59: 4.359999

### Appendix: with heapq-module

In [39]:
from heapq import heapify, heappush, heappop

def dijkstra_hq(N, L, origin):
        INF = 10 ** 9
        
        num_node = len(N)
        num_link = len(L)
    
        spl  = np.ones(shape=num_node)*INF#shortest path length
        flag = np.zeros(shape=num_node)
    
        hq = [(0, origin)] # (distance, node)
        spl[N[origin]['index']]  = 0
    
        while hq:
            from_node = heappop(hq)[1] # 確定させるノードを pop する
            
            if flag[N[from_node]['index']] == 2:
                continue #探索済ならスキップ
            else:
                for l in N[from_node]['outlink_list']:
                    to_node = L[l]['to_node']
                    tmp_cost = spl[N[from_node]['index']] + L[l]['ff_time']
            
                    if flag[N[to_node]['index']] == 2:
                        continue #探索済ならスキップ
                    else:
                        if tmp_cost < spl[N[to_node]['index']]:
                            spl[N[to_node]['index']]  = tmp_cost
                            flag[N[to_node]['index']] = 1 #探索中に変更
                            heappush(hq, (tmp_cost, to_node)) #ヒープに追加
                
                flag[N[from_node]['index']] = 2 #探索済に変更 
        
        return spl

In [40]:
%%time
dijkstra_hq(N, L, 1)

Wall time: 176 ms


array([0.000e+00, 1.000e+09, 3.600e+00, ..., 4.187e+01, 6.110e+00,
       1.126e+01])